# MASA: Howto Mini App Streaming Source (MASA)

In [1]:
%%capture
# System Libraries
import sys, os
sys.path.insert(0, "..")
import pandas as pd

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("radical.utils").setLevel(logging.ERROR)
 
# Pilot-Streaming
import pilot.streaming
import masa.spark

Start Spark Streaming Cluster

In [4]:
spark_pilot_description = {
    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 159,
    "type":"spark"
}
spark_pilot = pilot.streaming.PilotComputeService.create_pilot(spark_pilot_description)
spark_pilot.wait()

**** Job: 60703 State : Pending
Create Spark Context for URL: spark://129.114.58.122:7077
Create Spark Context for URL: spark://129.114.58.122:7077


Run Mini-App

In [2]:
import os, sys
sys.path.insert(0, "..")
import masa.spark
spark_details = {'master_url':'spark://129.114.58.122:7077'}
kafka_details = {'master_url':'c251-121:2181'}
streaming_analytics =  masa.spark.MiniApp(spark_master=spark_details["master_url"],
                             kafka_zk_hosts=kafka_details["master_url"],
                             topic_name = "light_test8",
                             test_scenario="test",
                             application = "light"
                            )

/home/01131/tg804093/work/kafka_2.11-1.0.0/bin/kafka-topics.sh --describe --topic light_test8 --zookeeper c251-121:2181
Streaming Window: 60 sec


In [3]:
streaming_analytics.run_in_background()

spark-submit --master spark://129.114.58.122:7077 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.1 --conf 'spark.executor.memory=110g' /home/01131/tg804093/notebooks/streaming-miniapps/masa/spark.py spark://129.114.58.122:7077 c251-121:2181 light_test8 test 60 light


In [5]:
streaming_analytics.cancel()